<a href="https://colab.research.google.com/github/Federico6419/MachineLearningProject/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch import Tensor
from collections import deque
import config_test
from model import Model
import matplotlib.pyplot as plt
import cv2

#Average rewards
average_random = np.zeros(config_test.NUM_EPISODES)
average_QTable = np.zeros(config_test.NUM_EPISODES)
average_nn = np.zeros(config_test.NUM_EPISODES)

#Cumulative reward
cum_reward_random = np.zeros(config_test.NUM_EPISODES)
cum_reward_QTable = np.zeros(config_test.NUM_EPISODES)
cum_reward_nn = np.zeros(config_test.NUM_EPISODES)

#Action Space
action_space = [
                (-1, 1, 0.2), (0, 1, 0.2), (1, 1, 0.2),
                (-1, 1,   0), (0, 1,   0), (1, 1,   0),
                (-1, 0, 0.2), (0, 0, 0.2), (1, 0, 0.2),
                (-1, 0,   0), (0, 0,   0), (1, 0,   0)
              ]

#This is the function that discretizes the image
def discretize_image(image):
    image = image.unsqueeze(0)

    #Execute the 3x3 Average Pooling
    pooling = nn.AvgPool2d(8, stride=8)
    image = pooling(image)
    image = image.squeeze(0)

    #Create the image identifier
    string = "S"
    for i, x in enumerate(image.numpy()):
        for j in x:
            if(j < 30):
                string = string + "0"
            elif(j < 60):
                string = string + "1"
            elif(j < 90):
                string = string + "2"
            elif(j < 120):
                string = string + "3"
            elif(j < 150):
                string = string + "4"
            elif(j < 180):
                string = string + "5"
            elif(j < 210):
                string = string + "6"
            elif(j < 256):
                string = string + "7"

    return string


#Initialize the Model and load weights
model = Model().to(config_test.DEVICE)
optimizer = optim.Adam(model.parameters(), lr=config_test.LR)
config_test.load_model(config_test.CHECKPOINT, model, optimizer)

#Load the Q-Table csv file
Q_Table = np.loadtxt('QTable.csv', delimiter=',')

#Policies
def policy(p, state):
    if(p == "Random"):
        return action_space[random.randrange(len(action_space))]          #Sample a random action from the Action Space

    elif(p == "Q-Table"):
        if state in Q_Table:
            return action_space[np.argmax(Q_Table[state])]                #Return the action with the highest value for the current state in the Q-Table
        else:
            return (0, 1, 0)

    elif(p == "nn"):
        prediction = model(torch.from_numpy(state.astype('float32')).to(config_test.DEVICE)).detach().cpu().numpy()
        action = action_space[np.argmax(prediction)]              #Select the action with the maximum predicted Q-Value
        return action


#Initialize the Car Racing Environment
env = gym.make("CarRacing-v2", render_mode="human")
observation, info = env.reset()

#Testing
def test(strategy):
    c_reward = 0                #Cumulative reward
    total_reward = 0
    episode = 0
    for i in range(config_test.NUM_EPISODES):
            observation, info = env.reset()

            observation = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)           #Convert the state into a Grayscale Image, that is a Matrix 96x96 composed by Integer values

            if(strategy == "Neural Network policy"):
                frames_queue = deque([observation]*3, maxlen = 3)
            elif(strategy == "Q-Table policy"):
                #Discretize the image
                observation = torch.from_numpy(observation.astype('float32'))
                observation = discretize_image(observation)

            for _ in range(250):
                env.render()

                if(strategy == "Random policy"):
                    action = policy("Random", observation)
                elif(strategy == "Q-Table policy"):
                    observation = discretize_image(observation)
                    action = policy("Q-Table", observation)
                elif(strategy == "Neural Network policy"):
                    current_frame = np.array(frames_queue)
                    action = policy("nn", current_frame)

                observation, reward, done, truncated, info = env.step(action)
                c_reward += reward

                observation = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)

                if(strategy == "Neural Network policy"):
                    frames_queue.append(observation)

                if done or truncated:
                    total_reward += c_reward
                    break

            if(strategy == "random policy"):
                cum_reward_random[episode] = c_reward # save the cum_reward for the relative episode
                average_random[episode]=total_reward/episode
            elif(strategy == "q table policy"):
                cum_reward_table[episode] = c_reward
                average_table[episode]=total_reward/episode
            elif(strategy == "neural network policy"):
                cum_reward_nn[episode] = c_reward
                average_nn[episode]=total_reward/episode

            print("The cumulative reward is:",c_reward)
            c_reward = 0 # reset the current cumulative reward
            print("Episode: ",episode)
            episode += +1

    print(f"Tests for {strategy} finished after {config_test.NUM_EPISODES} episodes")



test("Random policy")
test("Q-Table policy")
test("Neural Network policy")

env.close()